In [ ]:
import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning, module='anndata')
warnings.filterwarnings('ignore', message='.*reticulate.*')

In [ ]:
import os
import rpy2
import logging
import warnings
import anndata2ri
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
from itertools import chain
from datetime import datetime
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from scipy.sparse import csr_matrix
from joblib import Parallel, delayed
from functions import helper_functions
from matplotlib.pyplot import rcParams
from functions.run_nebula import run_de_tests, build_effect_size_anndata, get_standardized_effects




In [3]:
# # Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

-----
anndata     0.11.1
scanpy      1.9.3
-----
PIL                         9.5.0
anndata2ri                  1.1
asttokens                   NA
backcall                    0.2.0
cffi                        1.15.1
comm                        0.1.3
cycler                      0.10.0
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.6.7
decorator                   5.1.1
decoupler                   1.4.0
dot_parser                  NA
exceptiongroup              1.1.1
executing                   1.2.0
functions                   NA
google                      NA
gsva_prep                   NA
h5py                        3.9.0
igraph                      0.10.4
ipykernel                   6.23.2
ipywidgets                  8.0.6
jedi                        0.18.2
jinja2                      3.1.2
joblib                      1.2.0
kiwisolver                  1.4.4
leidenalg                   0.9.1
llvmlite                    0.39.1
louvain      

In [4]:
%%R

suppressPackageStartupMessages({
    library(Matrix)
    library(viridis)
    library(harmony)
    library(ggpubr)
    library(tictoc)
    library(RColorBrewer)
    library(Hmisc)
    library(corrplot)
    library(grid)
    library(gridExtra)
    library(igraph)
    library(ggrepel)
    library(readxl)
    library(conflicted)
    library(dplyr)
    library(parallel)
    library(stringr)
    library(tibble)
    library(BiocParallel)

    # single-cell analysis package
    library(Seurat)
    library(zellkonverter)   
    library(SingleCellExperiment)
    library(tidyr)
    library(readxl)
    library(GSA)
    library(limma)

    # plotting and data science packages
    library(tidyverse)
    library(cowplot)
    library(patchwork)
    library(ggplot2)

    # co-expression network analysis packages:
    library(WGCNA)
    library(hdWGCNA)

    # gene enrichment packages
    library(enrichR)
    library(GeneOverlap)
    library(GSEABase)
    library(GSVA) 

    # cell-cell communication
    library(nichenetr)

# needs to be run every time you start R and want to use %>%
})

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

# optionally enable multithreading
enableWGCNAThreads(nThreads = 40)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    Allowing parallel execution with up to 40 working processes.


In addition: Warning messages:
1: replacing previous import ‘GenomicRanges::intersect’ by ‘SeuratObject::intersect’ when loading ‘hdWGCNA’ 
2: replacing previous import ‘GenomicRanges::union’ by ‘dplyr::union’ when loading ‘hdWGCNA’ 
3: replacing previous import ‘GenomicRanges::setdiff’ by ‘dplyr::setdiff’ when loading ‘hdWGCNA’ 
4: replacing previous import ‘dplyr::as_data_frame’ by ‘igraph::as_data_frame’ when loading ‘hdWGCNA’ 
5: replacing previous import ‘Seurat::components’ by ‘igraph::components’ when loading ‘hdWGCNA’ 
6: replacing previous import ‘dplyr::groups’ by ‘igraph::groups’ when loading ‘hdWGCNA’ 
7: replacing previous import ‘dplyr::union’ by ‘igraph::union’ when loading ‘hdWGCNA’ 
8: replacing previous import ‘GenomicRanges::subtract’ by ‘magrittr::subtract’ when loading ‘hdWGCNA’ 
9: replacing previous import ‘Matrix::as.matrix’ by ‘proxy::as.matrix’ when loading ‘hdWGCNA’ 
10: replacing previous import ‘igraph::groups’ by ‘tidygraph::groups’ when loading ‘hdWGCNA’ 

## **Data Prep Parameters**

- `test_names`: List of the different test names of interest.

- `save_prefix`: Preferred prefix for saving critical files. Ideally chosen to be in the format `{source name}_{brain region}`. e.g `mathys_pfc`

- `subject_id`: Column name for Subject/Patient ID in both metadata and `.obs`

In [11]:
save_prefix = 'seaad_mtg'                                       # this takes the format '{StudyName}_{ThreeLetterAccronymForBrainRegion}'
subject_id = helper_functions.clean_strings('Donor ID')
cell_type_column = 'Subclass'                       # 'Supertype (non-expanded)', 'Subclass'
pseudobulk_level = 'Supertype'                      # Level at which cells are aggregated into pseudobulks,
factor = 'Continuous Pseudo-progression Score'    # pathology.group # Continuous Pseudoprogression Score
factor = helper_functions.clean_strings(factor, replace_hyphen=True)
test_names = ['late_vs_early']                                  # test categories
region_name = save_prefix.split('_')[-1].upper()
count_agg_strategy = 'metacell'   # options 'network', 'random', 'pseudobulk', 'smaller_network', 'standardbulk', 'metacell', `blanchardbulk``


data_dir = f'/media/tadeoye/Volume1/data/seq/SEA-AD/{region_name}/RNAseq/'
save_dir = f'../results/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

subclass = {
    'excitatory': ['L5 IT', 'L2/3 IT', 'L4 IT', 'L6 IT', 'L6 IT Car3', 'L5/6 NP', 'L6b', 'L6 CT', 'L5 ET'],
    'inhibitory': ['Pvalb', 'Sst', 'Lamp5 Lhx6', 'Vip', 'Lamp5', 'Sncg', 'Chandelier', 'Sst Chodl', 'Pax6'],
    'astrocyte': ['Astrocyte'],
    'microglia': ['Microglia-PVM'],
    'opc': ['OPC'],
    'oligodendrocyte': ['Oligodendrocyte'],
    'endothelial': ['Endothelial'],
    'vlmc': ['VLMC'],
    }

cell_supertypes = list(chain(*list(subclass.values())))



# **Load Merged Pseudo-bulked Data**

We have merged the pseduobulked data for all cell sypertypes (`02_merge_pseudobulks.ipynb`). Now, we load the merged data.

In [6]:
clean_subclass = robjects.ListVector({k: [helper_functions.clean_strings(v, preserve_case=True) for v in val] for k, val in subclass.items()})
clean_cell_supertypes = [helper_functions.clean_strings(cell_type, preserve_case=True) for cell_type in cell_supertypes]

In [7]:
adata_merged = sc.read_h5ad(data_dir+f'counts/anndata/all_subclass_{count_agg_strategy}_anndata.h5ad')
adata_merged.X = csr_matrix(adata_merged.X)

adata_annot = sc.read_h5ad(os.path.join(data_dir, f'counts/anndata/{save_prefix.upper()}_RNAseq_final-nuclei.2024-02-13.h5ad'))
adata_annot.X = adata_annot.layers['UMIs'] 

Subclass = robjects.ListVector(subclass)

In [8]:
%%R -i adata_merged -i count_agg_strategy -i data_dir -i subject_id -i Subclass -i cell_supertypes -i clean_cell_supertypes -i clean_subclass

source('../scripts/functions/helper_functions.r')

print('Merged data loaded')
adata_merged

[1] "Merged data loaded"
class: SingleCellExperiment 
dim: 36601 111258 
metadata(0):
assays(2): X logcounts
rownames(36601): MIR1302-2HG FAM138A ... AC007325.4 AC007325.2
rowData names(0):
colnames(111258): L5 IT_1#H19.33.004_1 L5 IT_1#H19.33.004_2 ...
  VLMC_1#H21.33.027_5 VLMC_1#H21.33.027_6
colData names(21): donor_id orig.ident ... pseudo_genes_detected
  pseudo_number_of_umis
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):


In [9]:
# del adata_merged, summed_counts_per_celltype

In [13]:
n_cores = 40

covariates = [
    'Continuous Pseudo-progression Score',
    'Age at Death binned codes',
    'Sex',
    'APOE4_Status',
    'Pseudo Genes Detected',
    'Pseudo Number of UMIs',
    # 'PMI',
    'method',
    'Race (choice=White)',
]

covariates = helper_functions.clean_strings(covariates, replace_hyphen=True)

In [ ]:

if run_differential_expression:
    # Set up parameters
    params = {
        'region': 'MTG',
        'target': '',
        'split_key': pseudobulk_level,
        'outgroup_de': False, 
        'outgroup': '', 
        'covariates': covariates,
        'random_effect': subject_id,
        'covariate_formula': ' + '.join(covariates[1:]) + " + ",
        'tests': [factor],
        'layer': 'X',  
        'offset_variable': helper_functions.clean_strings('Pseudo Number of UMIs'),
        'group_cell': False,
        'save_dir': data_dir+'results/differential_gene_expression'  
    }

    adata_merged.obs[params['split_key']] = adata_merged.obs[params['split_key']].astype('category')

    os.makedirs(params['save_dir'], exist_ok=True)

    def process_target(target, adata_merged, params, cell_type_column):
        adata = adata_merged[adata_merged.obs[cell_type_column] == target]
        results = []
        for b in adata.obs[params['split_key']].cat.categories:
            try:
                result = run_de_tests(
                    adata_merged,
                    params['region'],
                    helper_functions.clean_strings(target,preserve_case=True,replace_hyphen=False),
                    params['split_key'],
                    params['outgroup_de'],
                    b,
                    params['covariates'],
                    params['random_effect'],
                    params['covariate_formula'],
                    params['tests'],
                    params['layer'],
                    params['offset_variable'],
                    params['group_cell'],
                    params['save_dir']
                )
                results.append(result)
            except Exception as e:
                error_msg = str(e)
                if "Some predictors have zero variation or a zero vector" in error_msg:
                    print(f"{datetime.now()} -- Skipping {b} because some predictors have zero variation or a zero vector")
                else:
                    print(f"{datetime.now()} -- Error processing {b}: {str(e)}") 
                continue
                
        return results

    # Parallelize at target level
    targets = adata_merged.obs[cell_type_column].unique()
    results = Parallel(n_jobs=n_cores)(
        delayed(process_target)(target, adata_merged, params, cell_type_column) 
        for target in targets
    )

/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    
    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/home/tadeoye/miniconda3/envs/

...

2025-01-22 02:02:18.459190 -- Pvalb_9 along continuous_pseudo_progression_score was written to disk
2025-01-22 02:02:19.352399 -- No outgroup specified, running analysis for all groups and tests
2025-01-22 02:02:19.487096 -- Removing 5899 features from Pvalb_10 for low numbers of counts per cell
2025-01-22 02:02:19.515304 -- Detected continuous_pseudo_progression_score as an integer or float, applying a min-max normalization
2025-01-22 02:02:19.542853 -- Adding 3 pseudocounts to continuous_pseudo_progression_score=0.707882151477128 for 2460 genes in Pvalb_10 because all values are 0 for that covariate
2025-01-22 02:11:04.181711 -- Adding 3 pseudocounts to continuous_pseudo_progression_score=0.0 for 779 genes in Pvalb_10 because all values are 0 for that covariate
2025-01-22 02:13:50.984269 -- Detected age_at_death_binned_codes as an integer or float, applying a min-max normalization
2025-01-22 02:13:51.049825 -- Removing sex from the covariate formula
2025-01-22 02:13:51.049937 -- Remo

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In nebula(counts, obs$donor_id, pred = df, offset = obs$pseudo_number_of_umis,  :
R[write to console]: 
 
R[write to console]:  Some predictors in the design matrix are collinear or linearly dependent. The effects of these predictors are not identifiable.



2025-01-22 02:15:38.011543 -- Pvalb_10 along continuous_pseudo_progression_score was written to disk
2025-01-22 02:15:38.962814 -- No outgroup specified, running analysis for all groups and tests
2025-01-22 02:15:39.583940 -- Removing 5376 features from Pvalb_12 for low numbers of counts per cell
2025-01-22 02:15:39.706340 -- Detected continuous_pseudo_progression_score as an integer or float, applying a min-max normalization
2025-01-22 02:15:39.788640 -- Adding 3 pseudocounts to continuous_pseudo_progression_score=0.0 for 1667 genes in Pvalb_12 because all values are 0 for that covariate
2025-01-22 02:32:51.799059 -- Adding 3 pseudocounts to continuous_pseudo_progression_score=0.4145860536076113 for 1313 genes in Pvalb_12 because all values are 0 for that covariate
2025-01-22 02:46:22.137995 -- Adding 3 pseudocounts to continuous_pseudo_progression_score=0.6666314547467305 for 3811 genes in Pvalb_12 because all values are 0 for that covariate
2025-01-22 02:49:20.826863 -- Skipping L5_

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In nebula(counts, obs$donor_id, pred = df, offset = obs$pseudo_number_of_umis,  :
R[write to console]: 
 
R[write to console]:  Some predictors in the design matrix are collinear or linearly dependent. The effects of these predictors are not identifiable.



2025-01-22 04:28:47.188093 -- Pvalb_14 along continuous_pseudo_progression_score was written to disk
2025-01-22 04:28:48.020374 -- No outgroup specified, running analysis for all groups and tests
2025-01-22 04:28:50.500477 -- Removing 4636 features from Pvalb_15 for low numbers of counts per cell
2025-01-22 04:28:50.981817 -- Detected continuous_pseudo_progression_score as an integer or float, applying a min-max normalization
2025-01-22 04:28:50.983095 -- Detected age_at_death_binned_codes as an integer or float, applying a min-max normalization
2025-01-22 04:28:51.334298 -- Adding 3 pseudocounts to age_at_death_binned_codes=0.75 for 75 genes in Pvalb_15 because all values are 0 for that covariate
2025-01-22 04:32:19.825550 -- Adding 3 pseudocounts to age_at_death_binned_codes=1.0 for 330 genes in Pvalb_15 because all values are 0 for that covariate
2025-01-22 04:47:43.233803 -- Adding 3 pseudocounts to age_at_death_binned_codes=0.25 for 734 genes in Pvalb_15 because all values are 0 f

In [173]:
blacklisted_genes = ["MTRNR2L12", "TTTY14", "USP9Y", "NLGN4Y", "UTY", "DDX3Y", "KDM5D", "TTTY10"]
blacklisted_genes.extend(adata_merged.var_names[adata_merged.var_names.str.startswith("MT-")].to_list())


effect_sizes, pvalues, std_errors  = build_effect_size_anndata(
    results_dir=os.path.join(data_dir+'results/differential_gene_expression',),
    glob_pattern="*.csv",
    file_pattern="_across_continuous_pseudo_progression_score_DE.csv",
    test="continuous_pseudo_progression_score",
    adata=adata_merged,
    subclass=cell_type_column,
    celltype=cell_type_column,
    blacklisted_genes=blacklisted_genes,
)

NameError: name 'build_effect_size_anndata' is not defined

In [174]:
effect_size_table = pd.read_csv('/media/tadeoye/Volume1/data/seq/SEA-AD/MTG/supplementary/Nebula Results/effect_size_table.csv')
effect_size_table

,Unnamed: 0,Gene,Taxonomy Level,Population,Effect size across all of pseudoprogression,Effect size across early pseudoprogression,Effect size across late pseudoprogression,Mean expression (natural log UMIs per 10k plus 1)
0,0,MIR1302-2HG,Supertype,Lamp5_Lhx6_1,0.00,0.00,0.00,0.00
1,1,FAM138A,Supertype,Lamp5_Lhx6_1,0.00,0.00,0.00,0.00
2,2,OR4F5,Supertype,Lamp5_Lhx6_1,0.00,0.00,0.00,0.00
3,3,AL627309.1,Supertype,Lamp5_Lhx6_1,-2.17,-0.79,0.28,0.01
4,4,AL627309.3,Supertype,Lamp5_Lhx6_1,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...
6072275,6072275,AC141272.1,Class,Non-neuronal and non-neural,0.00,0.00,0.00,0.00
6072276,6072276,AC023491.2,Class,Non-neuronal and non-neural,0.00,0.00,0.00,0.00
6072277,6072277,AC007325.1,Class,Non-neuronal and non-neural,0.03,0.05,0.04,0.00
6072278,6072278,AC007325.4,Class,Non-neuronal and non-neural,0.47,-0.04,0.29,0.02


In [ ]:
results_dir = os.path.join(data_dir+'results/differential_gene_expression',
                             subclass.capitalize(),
                             factor, 
                             f'{subclass.capitalize()}_{subclass.capitalize()}_across_{factor}_DE.csv')

effect_sizes = get_standardized_effects(results_dir, factor, filter_genes=False, thr = 0.01)